# A Gentle Introduction to Optimization 
---

## Capítulo 1

In [1]:
using Pkg
pkg"activate ../."
pkg"instantiate"

 Activating environment at `~/OneDrive/FGV/7 Período/Programação Linear/Project.toml`


In [2]:
pkg"add JuMP GLPK Ipopt"
using Plots, LinearAlgebra, JuMP, GLPK, Ipopt 

   Updating registry at `~/.julia/registries/General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


Fetching: [========================================>]  100.0 %

  Resolving package versions...
No Changes to `~/OneDrive/FGV/7 Período/Programação Linear/Project.toml`
No Changes to `~/OneDrive/FGV/7 Período/Programação Linear/Manifest.toml`


### Compra de petróleo KWOil

KWOil é um fornecedor local de óleo para aquecimento. A empresa existe há muitos anos e conhece o seu território. Em particular, KWOil desenvolveu um modelo confiável para prever a demanda futura de petróleo. Para cada um dos quatro meses seguintes, a empresa espera as seguintes quantidades de demanda de óleo para aquecimento. Na tabela a seguir temos as informações da demanda a cada mês estimada pelo modelo da empresa e o preço de compra por litro de um fornecedor regional. 

|Mês|1|2|3|4|
|---|-|-|-|-|
|Demanda|5000|8000|9000|6000|
|Preço de compra|0.75|0.72|0.92|0.9|

A empresa pode estocar no máximo 4000 L e no ínicio do mês 1 já tem 2000L estocados. Se empresa compra petróleo no mês $i$, pode enviar diretamente ao cliente sem precisar armazenar. Assim, supondo que a empresa queira minimizar o custo de compra da regional e cobrir a demanda, quanto deve comprar por mês?


Defina as variáveis 

$$
p_i = \text{quantidade de litros comprados no inicio do mes } i
$$
$$
t_i = \text{quantidade de litros armazenados no inicio do mes } i
$$
Queremos 
$$
\min 0.75p_1 + 0.72p_2 + 0.92p_3 + 0.9p_4
$$
sujeito a 
$$
p_1 + t_1 = 5000 + t_2
$$
$$
p_2 + t_2 = 8000 + t_3
$$
$$
p_3 + t_3 = 9000 + t_4
$$
$$
p_4 + t_4 \ge 6000
$$
$$
t_1 = 2000, 0 \le t_2, t_3, t_4 \le 4000
$$
$$
p_1,...,p_4 \ge 0
$$

In [12]:
# Modelo e Solver
model = Model(GLPK.Optimizer)

# Variaveis, canalizações (ou caixas) e tipo
@variable(model,p1 >= 0)
@variable(model,p2 >= 0)
@variable(model,p3 >= 0)
@variable(model,p4 >= 0)
@variable(model,4000 >= t2 >= 0)
@variable(model,4000 >= t3 >= 0)
@variable(model,4000 >= t4 >= 0)

# Restrições
@constraint(model, p1 - t2 == 3000)
@constraint(model, p2 + t2 - t3 == 8000)
@constraint(model, p3 + t3 - t4 == 9000)
@constraint(model, p4 + t4 >= 6000)

# Função objetivo
@objective(model,Min,0.75p1 + 0.72p2 + 0.92p3 + 0.9p4)

print(model)

Min 0.75 p1 + 0.72 p2 + 0.92 p3 + 0.9 p4
Subject to
 p1 - t2 = 3000.0
 p2 + t2 - t3 = 8000.0
 p3 + t3 - t4 = 9000.0
 p4 + t4 ≥ 6000.0
 p1 ≥ 0.0
 p2 ≥ 0.0
 p3 ≥ 0.0
 p4 ≥ 0.0
 t2 ≥ 0.0
 t3 ≥ 0.0
 t4 ≥ 0.0
 t2 ≤ 4000.0
 t3 ≤ 4000.0
 t4 ≤ 4000.0


In [13]:
# Chamada do Solver
optimize!(model)

#Declarar solução
@show value(p1)
@show value(p2)
@show value(p3)
@show value(p4)
@show value(t2)
@show value(t3)
@show value(t4)

@show objective_value(model)

value(p1) = 3000.0
value(p2) = 12000.0
value(p3) = 5000.0
value(p4) = 6000.0
value(t2) = 0.0
value(t3) = 4000.0
value(t4) = 0.0
objective_value(model) = 20890.0


20890.0

### O problema da atribuição

A empresa Watertech enfrenta o seguinte problema: há um conjunto de $J$ jobs que precisam ser tratados com urgência. A empresa selecionou $I$ de seus funcionários de maior confiança para realizar essas tarefas. Naturalmente, os conjuntos de habilidades desses funcionários diferem, e nem todos os trabalhos são igualmente bem administrados por todos os funcionários. Com base em experiências anteriores, a administração sabe quantas horas $c_{ij}$ cada trabalhador $i \in I$ deve levar para concluir qualquer uma das tarefas $j \in J$. A tabela a seguir fornece um exemplo de um caso com $| J | = 4$ jobs e $| I | = 4$ funcionários:

|Funcionários/Jobs|1|2|3|4|
|------------|-|-|-|-|
|1|3|5|1|7|
|2|8|2|2|4|
|3|2|1|6|8|
|4|8|3|3|2|

WaterTech quer atribuir os jobs aos empregados com a seguinte condição: 

1. cada empregado $i \in I$ é atribuído um job $j \in J$. 
2. cada trabalho $j \in J$ é atribuído a um empregado $i \in I$. 

Para isso vamos criar as **variáveis** $x_{ij}$ binárias que indicam 1 se o funcionário $i$ teve atribuído o job $j$ e 0 caso contrário. 

Queremos, então, que para todo $i \in I$, 
$$
\sum_{j \in J} x_{ij} = 1
$$
e para todo $j \in J$, 
$$
\sum_{i \in I} x_{ij} = 1
$$
Além disso temos a restrição que 
$$
0 \le x_{ij} \le 1 \text{ e } x_{ij} \in \mathbb{Z}
$$

Agora, nosso **funcional objetivo** será
$$
\min \sum_{i \in I}\sum_{j \in J} c_{ij}x_{ij}
$$

In [12]:
# definindo tamanho de I e J
N = 4
assignment = Model(GLPK.Optimizer)

# Variáveis: estou decladando elas como binárias 
@variable(assignment, x[i=1:N,j=1:N], Bin) 

# Restrições 
@constraint(assignment, [sum(x[i,:]) for i=1:N] .== 1)
@constraint(assignment, [sum(x[:,j]) for j=1:N] .== 1)

# Função objetivo 
c = [3 5 1 7; 
     8 2 2 4;
     2 1 6 8; 
     8 3 3 2];

@objective(assignment, Min, sum(c.*x));

print(assignment)

Min 3 x[1,1] + 8 x[2,1] + 2 x[3,1] + 8 x[4,1] + 5 x[1,2] + 2 x[2,2] + x[3,2] + 3 x[4,2] + x[1,3] + 2 x[2,3] + 6 x[3,3] + 3 x[4,3] + 7 x[1,4] + 4 x[2,4] + 8 x[3,4] + 2 x[4,4]
Subject to
 x[1,1] + x[1,2] + x[1,3] + x[1,4] = 1.0
 x[2,1] + x[2,2] + x[2,3] + x[2,4] = 1.0
 x[3,1] + x[3,2] + x[3,3] + x[3,4] = 1.0
 x[4,1] + x[4,2] + x[4,3] + x[4,4] = 1.0
 x[1,1] + x[2,1] + x[3,1] + x[4,1] = 1.0
 x[1,2] + x[2,2] + x[3,2] + x[4,2] = 1.0
 x[1,3] + x[2,3] + x[3,3] + x[4,3] = 1.0
 x[1,4] + x[2,4] + x[3,4] + x[4,4] = 1.0
 x[1,1] binary
 x[2,1] binary
 x[3,1] binary
 x[4,1] binary
 x[1,2] binary
 x[2,2] binary
 x[3,2] binary
 x[4,2] binary
 x[1,3] binary
 x[2,3] binary
 x[3,3] binary
 x[4,3] binary
 x[1,4] binary
 x[2,4] binary
 x[3,4] binary
 x[4,4] binary


In [13]:
# Utilizando o solver para resolver 
optimize!(assignment)

A solução diz que o primeiro funcionário deverá realizar o trabalho 3, o segundo o trabalho 2, o terceiro o trabalho 1 e o quatro o trabalho 4. 

In [18]:
Int.(value.(x))

4×4 Array{Int64,2}:
 0  0  1  0
 0  1  0  0
 1  0  0  0
 0  0  0  1